In [1]:
import cv2
import numpy as np
import glob
import os
import csv
import pandas as pd
from tqdm import tqdm
import warnings
from uncertainties import ufloat



In [2]:
# Path to the video file
video_path = r'_raw\_vids\300\300oprava.mpg'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Initialize variables to store the sum of frames and the frame count
frame_sum = None
frame_count = 0

# Get the total number of frames in the video
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Initialize the progress bar
progress_bar = tqdm(total=total_frames, desc="Processing frames")

# Loop through the video frames
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to float32 for accurate summation
    frame = frame.astype(np.float32)

    # Initialize frame_sum with the first frame
    if frame_sum is None:
        frame_sum = np.zeros_like(frame)

    # Accumulate the sum of frames
    frame_sum += frame
    frame_count += 1

    # Update the progress bar
    progress_bar.update(1)

# Close the progress bar
progress_bar.close()

# Calculate the average frame
if frame_count > 0:
    print ("Averaging frames...")
    average_frame = frame_sum / frame_count

    # Convert the average frame back to uint8
    print ("Converting to uint8...")
    average_frame = average_frame.astype(np.uint8)

    # Release the video capture object
    print ("Releasing video capture object...")
    cap.release()

    # Display the average frame
    print ("Displaying average frame")
    cv2.imshow('Average Frame', average_frame)

    # Save the average frame as an image
    cv2.imwrite('average_frame.png', average_frame)
else:
    print("Error: No frames read from the video file.")


NameError: name 'cv2' is not defined

In [4]:

# Read the first image to get properties
video_path = r'_raw\_vids\300\300oprava.mpg'
cap = cv2.VideoCapture(video_path)
# Get the width and height of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
current_frame = 0

# Read the first frame from the video
ret, prev_frame = cap.read()
if not ret:
    print("Error: Could not read the first frame from the video.")
    exit()

tracking_data_path = 'test0.csv'  # Replace with your desired CSV file path

# Define the codec and create VideoWriter object to save the processed video
output_path = 'test0.mp4'  # Replace with your output video file path
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file
fps = 30  # Frames per second
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Parameters for ShiTomasi corner detection
feature_params = dict(maxCorners=10000, qualityLevel=0.5, minDistance=30, blockSize=20)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.03))

# Take the first image and find corners in it
# Check if the first frame was read successfully
if prev_frame is None:
    print("Error: First frame is empty.")
    exit()

# Convert the first frame to grayscale
old_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Initialize the mask to zeros
# Ensure mask is of type CV_8UC1 and has the same size as old_gray
maskexc = np.ones_like(old_gray, dtype=np.uint8) * 255

# Set the mask to exclude a rectangle from 0-200px in both x and y directions (top-left corner which had a lot of unwanted artifacts)
#maskexc[0:307, 0:367] = 0

# Verify the mask
#assert maskexc.dtype == np.uint8, "Mask must be of type uint8"
#assert maskexc.shape == old_gray.shape, "Mask must have the same size as the input image"

# Detect initial points to track
p0 = cv2.goodFeaturesToTrack(old_gray, mask=maskexc, **feature_params)


cv2.destroyAllWindows()


# Create a mask image for drawing purposes
mask = np.zeros_like(old_gray)

# Initialize a dictionary to store tracking data
tracking_data = {}

# Initialize a dictionary to store total movement of each point
total_movement = {i: 0 for i in range(len(p0))}

# Initialize a dictionary to store the lifespan of each point
lifespan = {i: 0 for i in range(len(p0))}

# Initialize a list to store the original indexes of the points
original_indexes = list(range(len(p0)))

# Threshold for filtering stationary points
movement_threshold = 5.0  # Adjust this value as needed
min_distance_from_existing = 50  # Adjust this value as needed

# Minimum lifespan for points to be kept
min_lifespan = 100



In [5]:

progress_bar = tqdm(total=total_frames, desc="Processing frames")


# Loop through the image sequence
frame_idx = 0
while current_frame < total_frames:
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Update the progress bar
    progress_bar.update(1)
    # Calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None and st is not None and err is not None:
        good_new = p1[st == 1]
        good_old = p0[st == 1]
        good_indexes = [original_indexes[i] for i in range(len(st)) if st[i] == 1]

        # Calculate the distance moved by each point
        distances = np.linalg.norm(good_new - good_old, axis=1)

        # Update the total movement and lifespan for each point
        for i, dist in zip(good_indexes, distances):
            total_movement[i] += dist
            lifespan[i] += 1

        # Draw the tracks and save tracking data
        max_index = max(good_indexes) if good_indexes else 0
        frame_tracking_data = [None] * (2 * (max_index + 1))  # Initialize with None
        for i, (new, old) in zip(good_indexes, zip(good_new, good_old)):
            a, b = new.ravel()
            c, d = old.ravel()
            frame = cv2.circle(frame, (int(a), int(b)), 7, (0, 0, 200), -1)
    else:
        print(f"Optical flow calculation failed at frame {frame_idx}")

    

    # Update the total movement and lifespan for each point
    for i, dist in zip(good_indexes, distances):
        total_movement[i] += dist
        lifespan[i] += 1

    # Draw the tracks and save tracking data
    max_index = max(good_indexes) if good_indexes else 0
    frame_tracking_data = [None] * (2 * (max_index + 1))  # Initialize with None
    
    for i, (new, old) in zip(good_indexes, zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        frame = cv2.circle(frame, (int(a), int(b)), 4, (0, 0, 200), -1)
        frame = cv2.putText(frame, str(i), (int(a)+4, int(b)+4), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (10, 10, 10), 1, cv2.LINE_AA)
        frame_tracking_data[2*i] = a  # Set x coordinate
        frame_tracking_data[2*i+1] = b  # Set y coordinate

    img = cv2.add(frame, mask)

    # Write the processed frame to the output video
    out.write(img)

    # Store frame tracking data in the dictionary
    tracking_data[frame_idx] = frame_tracking_data

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = np.array(good_new).reshape(-1, 1, 2)
    original_indexes = good_indexes

    # Detect new points to track every 30 frames
    if frame_idx % 30 == 0:
        new_points = cv2.goodFeaturesToTrack(frame_gray, mask=maskexc, **feature_params)
        if new_points is not None:
            # Filter new points to ensure they are not close to existing points
            filtered_new_points = []
            for new_point in new_points:
                new_x, new_y = new_point.ravel()
                too_close = False
                for existing_point in p0:
                    existing_x, existing_y = existing_point.ravel()
                    distance = np.sqrt((new_x - existing_x)**2 + (new_y - existing_y)**2)
                    if distance < min_distance_from_existing:
                        too_close = True
                        break
                if not too_close:
                    filtered_new_points.append(new_point)
            
            if filtered_new_points:
                filtered_new_points = np.array(filtered_new_points).reshape(-1, 1, 2)
                p0 = np.vstack((p0, filtered_new_points))
                for i in range(len(filtered_new_points)):
                    total_movement[len(total_movement)] = 0
                    lifespan[len(lifespan)] = 0
                    original_indexes.append(len(original_indexes))
    cv2.imshow('frame', img)
    current_frame += 1
    frame_idx += 1

    
        
progress_bar.close()

# Release the video writer object
out.release()
cv2.destroyAllWindows()

# Filter out points based on total movement and lifespan
moving_points_indices = [i for i, total_dist in total_movement.items() if total_dist > movement_threshold and lifespan[i] >= min_lifespan]

# Filter tracking data to include only moving points
filtered_tracking_data = {}
for frame_idx, frame_data in tracking_data.items():
    filtered_frame_data = []
    for i in moving_points_indices:
        if 2*i+1 < len(frame_data) and frame_data[2*i] is not None and frame_data[2*i+1] is not None:
            filtered_frame_data.extend([frame_data[2*i], frame_data[2*i+1]])
        else:
            filtered_frame_data.extend([None, None])
    filtered_tracking_data[frame_idx] = filtered_frame_data

# Convert filtered tracking data dictionary to DataFrame
df_tracking = pd.DataFrame.from_dict(filtered_tracking_data, orient='index')

# Generate column names
num_points = df_tracking.shape[1] // 2
column_names = []
for i in range(num_points):
    column_names.append(f'Point_{i}_x')
    column_names.append(f'Point_{i}_y')

# Assign column names to DataFrame
df_tracking.columns = column_names

# Export tracking data to CSV
df_tracking.to_csv(tracking_data_path, index_label='Frame')

Processing frames:   0%|          | 0/3771 [00:00<?, ?it/s]

error: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:665: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


In [ ]:
results_df = pd.DataFrame(columns=['ID', 'r', 'ratio10/5', 'ratio15/10'])
file_path = 'Metoda3.csv'

# Suppress pandas PerformanceWarnings for better readability
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

# Constants
px = ufloat (53.3e-9,0.17e-9)
T = 295.05
R = 8.314
eta = 0.9589e-3
Na = 6.02214076e23

# Read the CSV file
df = pd.read_csv(file_path)
len0 = len(df.columns)


# Create new columns by subtracting the next row cell from the previous row cell for each column / Delta x/y
for col in df.columns:
    df[f'Delta_{col}'] = df[col].diff(-1)
    # Remove physically improbable outliers 
    df.loc[(df[f'Delta_{col}'] < -100) | (df[f'Delta_{col}'] > 100), f'Delta_{col}'] = np.nan

len1 = len(df.columns)

progresslen =( len1 - len0)/2
progress_bar = tqdm(total=progresslen, position=0, leave=False)
# Subtract every two columns
for i in range(len0, len(df.columns), 2):
    col1 = df.columns[i+1]
    col2 = df.columns[i + 2]
    # Calculate the RMS values for each pair of columns - this is RMS of the distance travelled in 150, 300 and 450 frames
    df[f'RMS150_{col1}/{col2}'] = (df[col1].rolling(window=150).sum()**2 + df[col2].rolling(window=150).sum()**2)
    df[f'RMS300_{col1}/{col2}'] = (df[col1].rolling(window=300).sum()**2 + df[col2].rolling(window=300).sum()**2)
    df[f'RMS450_{col1}/{col2}'] = (df[col1].rolling(window=450).sum()**2 + df[col2].rolling(window=450).sum()**2)

    #Drop the NaN values, calculate the mean and standard error of the mean for the RMS values for 150,300 and 450 respectively. Store as ufloat to be able to work with uncertainties
    if df[f'RMS150_{col1}/{col2}'].dropna().empty:
        df[f'FNM150{col1}/{col2}'] = np.nan
    else:
            mean_val = df[f'RMS150_{col1}/{col2}'].dropna().mean()
            std_err = df[f'RMS150_{col1}/{col2}'].dropna().sem()
            df[f'FNM150{col1}/{col2}'] = ufloat(mean_val, std_err)      

    if df[f'RMS300_{col1}/{col2}'].dropna().empty:
        df[f'FNM300{col1}/{col2}'] = np.nan
    else:
        mean_val = df[f'RMS300_{col1}/{col2}'].dropna().mean()
        std_err = df[f'RMS300_{col1}/{col2}'].dropna().sem()
        df[f'FNM300{col1}/{col2}'] = ufloat(mean_val, std_err)

    if df[f'RMS450_{col1}/{col2}'].dropna().empty:
        df[f'FNM450{col1}/{col2}'] = np.nan
    else:
        mean_val = df[f'RMS450_{col1}/{col2}'].dropna().mean()
        std_err = df[f'RMS450_{col1}/{col2}'].dropna().sem()
        df[f'FNM450{col1}/{col2}'] = ufloat(mean_val, std_err)
        
    df[f'ratio10/5{col1}/{col2}'] = df[f'FNM300{col1}/{col2}'] / df[f'FNM150{col1}/{col2}']
    df[f'ratio15/10{col1}/{col2}'] = df[f'FNM450{col1}/{col2}'] / df[f'FNM150{col1}/{col2}']

    # Check if the ratios are within the acceptable range and calculate the radius. Convert pixel values to meters print in nm for better readability
    if 1.85 <= df[f'ratio10/5{col1}/{col2}'].iloc[0] <= 2.15:
        if 2.75 <= df[f'ratio15/10{col1}/{col2}'].iloc[0] <= 3.25:
            df[f'r1-{col1}/{col2}'] = (((2*R*T*5)/(3*np.pi*eta*Na*df[f'FNM150{col1}/{col2}'].iloc[0]*(px**2))))
            df[f'r2-{col1}/{col2}'] = (((2*R*T*10)/(3*np.pi*eta*Na*df[f'FNM300{col1}/{col2}'].iloc[0]*(px**2))))
            df[f'r3-{col1}/{col2}'] = (((2*R*T*15)/(3*np.pi*eta*Na*df[f'FNM450{col1}/{col2}'].iloc[0]*(px**2))))
            print(f"Column ID: {col1}" ,"r1 = ", (df[f'r1-{col1}/{col2}'].iloc[0])*1e9, "nm", "r2 = ", (df[f'r2-{col1}/{col2}'].iloc[0])*1e9, "nm", "r3 = ", (df[f'r3-{col1}/{col2}'].iloc[0])*1e9, "nm")
            print(f"ratio for {col1} =", df[f'ratio10/5{col1}/{col2}'].iloc[0], "ratio2 for {col1} =", df[f'ratio15/10{col1}/{col2}'].iloc[0])

            # Calculate the radius - average of the three values
            r = (df[f'r1-{col1}/{col2}'].iloc[0] + df[f'r2-{col1}/{col2}'].iloc[0] + df[f'r3-{col1}/{col2}'].iloc[0])/3 * 1e9

            # Create a new dataframe to store the results
            results_df = pd.concat([results_df, pd.DataFrame({'ID': [col1], 'r': [r], 'ratio10/5': [df[f'ratio10/5{col1}/{col2}'].iloc[0]], 'ratio15/10': [df[f'ratio15/10{col1}/{col2}'].iloc[0]] })])

    progress_bar.update(1)
progress_bar.close()


# Display the contents of the CSV file
results_df

In [ ]:
import numpy as np

a0 = -70.74803
a1 = 5.169734
a2 = -0.1167653
a3 = 0.000839934
# Define the polynomial coefficients
coefficients = [a3, a2, a1, a0]

# Find the roots of the polynomial
roots = np.roots(coefficients)

# Display the roots
print("The roots of the polynomial are:", roots)



The roots of the polynomial are: [56.17942051+1.86965308j 56.17942051-1.86965308j 26.65839128+0.j        ]


: 

In [13]:
# Define the output directory for the split videos

video_path = r'_raw\_vids\600\0101_v1.mpg'
output_dir = 'splits\\600'
os.makedirs(output_dir, exist_ok=True)

# Initialize the frame counter
frame_counter = 0
segment_counter = 0

# Reopen the video file
cap = cv2.VideoCapture(video_path)

# Check if the video file opened successfully
if not cap.isOpened():
    print("Error: Could not open video file.")
    exit()

# Get the width and height of the video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Loop through the video frames
while True:
    frames = []
    for _ in range(60):
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        frame_counter += 1

    if not frames:
        break

    # Define the codec and create VideoWriter object to save the segment
    segment_path = os.path.join(output_dir, f'segment_{segment_counter}.mp4')
    out = cv2.VideoWriter(segment_path, fourcc, fps, (frame_width, frame_height))

    # Write the frames to the segment video
    for frame in frames:
        out.write(frame)

    # Release the VideoWriter object
    out.release()

    # Increment the segment counter
    segment_counter += 1
    print(f"Segment {segment_counter} saved.")


    # Set the video position back by 30 frames for overlapping
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter - 30)

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

Segment 1 saved.
Segment 2 saved.
Segment 3 saved.
Segment 4 saved.
Segment 5 saved.
Segment 6 saved.
Segment 7 saved.
Segment 8 saved.
Segment 9 saved.
Segment 10 saved.
Segment 11 saved.
Segment 12 saved.
Segment 13 saved.
Segment 14 saved.
Segment 15 saved.
Segment 16 saved.
Segment 17 saved.
Segment 18 saved.
Segment 19 saved.
Segment 20 saved.
Segment 21 saved.
Segment 22 saved.
Segment 23 saved.
Segment 24 saved.
Segment 25 saved.
Segment 26 saved.
Segment 27 saved.
Segment 28 saved.
Segment 29 saved.
Segment 30 saved.
Segment 31 saved.
Segment 32 saved.
Segment 33 saved.
Segment 34 saved.
Segment 35 saved.
Segment 36 saved.
Segment 37 saved.
Segment 38 saved.
Segment 39 saved.
Segment 40 saved.
Segment 41 saved.
Segment 42 saved.
Segment 43 saved.
Segment 44 saved.
Segment 45 saved.
Segment 46 saved.
Segment 47 saved.
Segment 48 saved.
Segment 49 saved.
Segment 50 saved.
Segment 51 saved.
Segment 52 saved.
Segment 53 saved.
Segment 54 saved.
Segment 55 saved.
Segment 56 saved.
S

In [15]:
import binascii

salt = "a7682b1ae5783053d9"
hash_value = "7fbdca4cf62192e6df921629bef4855b1dbf9f8be5d57ff65a67763f1c9197ef"

try:
    binascii.unhexlify(salt)  # Check if valid hex
    binascii.unhexlify(hash_value)  # Check if valid hex
    print("Hex format is correct!")
except binascii.Error:
    print("Invalid hex format! Check your values.")

Hex format is correct!
